In [1]:
import sys
import os
import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging
import _pickle as cPickle
import numpy as np
import pprint

model_file = './data/checkpoint_inception_v3/inception_v3_graph_def.pb'
input_description_file = './data/results.token' # 从这里获取每个文件的文件名
input_img_dir = './data/flickr30k-images/'
output_folder = './data/download_inpcetion_v3_features/'

batch_size = 1000 # 每个子文件中有1000张图片

if not gfile.Exists(output_folder):
    gfile.MakeDirs(output_folder)

def parse_token_file(token_file):
    """parse image description file"""
    img_name_to_tokens = {}
    with gfile.GFile(token_file, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        img_id, description = line.strip('\r\n').split('\t')
        image_name, _ = img_id.split('#')
        img_name_to_tokens.setdefault(image_name, [])
        img_name_to_tokens[image_name].append(description)
    return img_name_to_tokens

img_name_to_tokens = parse_token_file(input_description_file)
all_img_names = img_name_to_tokens.keys()

logging.info('num of all images: %d' % len(all_img_names))
pprint.pprint(list(all_img_names)[0:10])
pprint.pprint(img_name_to_tokens['1000268201.jpg'])



INFO:tensorflow:num of all images: 31783
['1000092795.jpg',
 '10002456.jpg',
 '1000268201.jpg',
 '1000344755.jpg',
 '1000366164.jpg',
 '1000523639.jpg',
 '1000919630.jpg',
 '10010052.jpg',
 '1001465944.jpg',
 '1001545525.jpg']
['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A little girl in a pink dress going into a wooden cabin .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl climbing into a wooden playhouse .',
 'A girl going into a wooden building .']


In [2]:
def load_pretrained_inception_v3(model_file):
    with gfile.GFile(model_file, 'rb') as f:
        # 建立一个空的计算图
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name = "")

load_pretrained_inception_v3(model_file)

In [6]:
num_batches = int(len(all_img_names) / batch_size)
if len(all_img_names) % batch_size != 0:
    num_batches += 1
    
with tf.Session() as sess:
    # 使用倒数第二层的特征
    second_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
    for i in range(num_batches):
        batch_img_names = list(all_img_names)[i*batch_size : (i+1)*batch_size]
        batch_features = []
        for img_name in batch_img_names:
            img_path = os.path.join(input_img_dir, img_name)
            logging.info('processing img %s' % img_name)
            if not gfile.Exists(img_path):
                continue
            print(img_path)
            img_data = gfile.GFile(img_path, 'rb').read()
            feature_vector = sess.run(second_to_last_tensor, 
                                     feed_dict = {
                                         "DecodeJpeg/contents:0": img_data
                                     })
            batch_features.append(feature_vector)
        batch_features = np.vstack(batch_features)
        output_filename = os.path.join(output_folder, "image_features_%d.pickle" % i)
        logging.info('writing to file %s' % output_filename)
        with gfile.GFile(output_filename, 'w') as f:
            cPickle.dump((batch_img_names, batch_features), f)
            

INFO:tensorflow:processing img 1000092795.jpg
./data/flickr30k-images/1000092795.jpg
INFO:tensorflow:processing img 10002456.jpg
./data/flickr30k-images/10002456.jpg
INFO:tensorflow:processing img 1000268201.jpg
./data/flickr30k-images/1000268201.jpg
INFO:tensorflow:processing img 1000344755.jpg
./data/flickr30k-images/1000344755.jpg
INFO:tensorflow:processing img 1000366164.jpg
./data/flickr30k-images/1000366164.jpg
INFO:tensorflow:processing img 1000523639.jpg
./data/flickr30k-images/1000523639.jpg
INFO:tensorflow:processing img 1000919630.jpg
./data/flickr30k-images/1000919630.jpg
INFO:tensorflow:processing img 10010052.jpg
./data/flickr30k-images/10010052.jpg
INFO:tensorflow:processing img 1001465944.jpg
./data/flickr30k-images/1001465944.jpg
INFO:tensorflow:processing img 1001545525.jpg
./data/flickr30k-images/1001545525.jpg
INFO:tensorflow:processing img 1001573224.jpg
./data/flickr30k-images/1001573224.jpg
INFO:tensorflow:processing img 1001633352.jpg
./data/flickr30k-images/100

./data/flickr30k-images/10287332.jpg
INFO:tensorflow:processing img 1028982826.jpg
./data/flickr30k-images/1028982826.jpg
INFO:tensorflow:processing img 1029450589.jpg
./data/flickr30k-images/1029450589.jpg
INFO:tensorflow:processing img 1029737941.jpg
./data/flickr30k-images/1029737941.jpg
INFO:tensorflow:processing img 1029802110.jpg
./data/flickr30k-images/1029802110.jpg
INFO:tensorflow:processing img 102998070.jpg
./data/flickr30k-images/102998070.jpg
INFO:tensorflow:processing img 1030041880.jpg
./data/flickr30k-images/1030041880.jpg
INFO:tensorflow:processing img 103031977.jpg
./data/flickr30k-images/103031977.jpg
INFO:tensorflow:processing img 1030985833.jpg
./data/flickr30k-images/1030985833.jpg
INFO:tensorflow:processing img 103106960.jpg
./data/flickr30k-images/103106960.jpg
INFO:tensorflow:processing img 103195344.jpg
./data/flickr30k-images/103195344.jpg
INFO:tensorflow:processing img 1031973097.jpg
./data/flickr30k-images/1031973097.jpg
INFO:tensorflow:processing img 10320

./data/flickr30k-images/1066252238.jpg
INFO:tensorflow:processing img 1066831905.jpg
./data/flickr30k-images/1066831905.jpg
INFO:tensorflow:processing img 106691539.jpg
./data/flickr30k-images/106691539.jpg
INFO:tensorflow:processing img 1067180831.jpg
./data/flickr30k-images/1067180831.jpg
INFO:tensorflow:processing img 1067675215.jpg
./data/flickr30k-images/1067675215.jpg
INFO:tensorflow:processing img 1067790824.jpg
./data/flickr30k-images/1067790824.jpg
INFO:tensorflow:processing img 1068427675.jpg
./data/flickr30k-images/1068427675.jpg
INFO:tensorflow:processing img 106959209.jpg
./data/flickr30k-images/106959209.jpg
INFO:tensorflow:processing img 107022557.jpg
./data/flickr30k-images/107022557.jpg
INFO:tensorflow:processing img 1071201387.jpg
./data/flickr30k-images/1071201387.jpg
INFO:tensorflow:processing img 1072153132.jpg
./data/flickr30k-images/1072153132.jpg
INFO:tensorflow:processing img 1072439304.jpg
./data/flickr30k-images/1072439304.jpg
INFO:tensorflow:processing img 1

INFO:tensorflow:processing img 109741554.jpg
./data/flickr30k-images/109741554.jpg
INFO:tensorflow:processing img 1097463464.jpg
./data/flickr30k-images/1097463464.jpg
INFO:tensorflow:processing img 109770830.jpg
./data/flickr30k-images/109770830.jpg
INFO:tensorflow:processing img 1097710887.jpg
./data/flickr30k-images/1097710887.jpg
INFO:tensorflow:processing img 109823394.jpg
./data/flickr30k-images/109823394.jpg
INFO:tensorflow:processing img 109823395.jpg
./data/flickr30k-images/109823395.jpg
INFO:tensorflow:processing img 109823397.jpg
./data/flickr30k-images/109823397.jpg
INFO:tensorflow:processing img 1099034049.jpg
./data/flickr30k-images/1099034049.jpg
INFO:tensorflow:processing img 1099434461.jpg
./data/flickr30k-images/1099434461.jpg
INFO:tensorflow:processing img 109982467.jpg
./data/flickr30k-images/109982467.jpg
INFO:tensorflow:processing img 1100214449.jpg
./data/flickr30k-images/1100214449.jpg
INFO:tensorflow:processing img 1100708949.jpg
./data/flickr30k-images/1100708

INFO:tensorflow:processing img 112550433.jpg
./data/flickr30k-images/112550433.jpg
INFO:tensorflow:processing img 112604461.jpg
./data/flickr30k-images/112604461.jpg
INFO:tensorflow:processing img 1126137905.jpg
./data/flickr30k-images/1126137905.jpg
INFO:tensorflow:processing img 1126403094.jpg
./data/flickr30k-images/1126403094.jpg
INFO:tensorflow:processing img 1126450961.jpg
./data/flickr30k-images/1126450961.jpg
INFO:tensorflow:processing img 112719695.jpg
./data/flickr30k-images/112719695.jpg
INFO:tensorflow:processing img 1128230658.jpg
./data/flickr30k-images/1128230658.jpg
INFO:tensorflow:processing img 1128874064.jpg
./data/flickr30k-images/1128874064.jpg
INFO:tensorflow:processing img 112945950.jpg
./data/flickr30k-images/112945950.jpg
INFO:tensorflow:processing img 1129704496.jpg
./data/flickr30k-images/1129704496.jpg
INFO:tensorflow:processing img 1130017585.jpg
./data/flickr30k-images/1130017585.jpg
INFO:tensorflow:processing img 1130295306.jpg
./data/flickr30k-images/113

...

./data/flickr30k-images/2349781135.jpg
INFO:tensorflow:processing img 2349785157.jpg
./data/flickr30k-images/2349785157.jpg
INFO:tensorflow:processing img 2350400382.jpg
./data/flickr30k-images/2350400382.jpg
INFO:tensorflow:processing img 2350565030.jpg
./data/flickr30k-images/2350565030.jpg
INFO:tensorflow:processing img 2350646648.jpg
./data/flickr30k-images/2350646648.jpg
INFO:tensorflow:processing img 235065283.jpg
./data/flickr30k-images/235065283.jpg
INFO:tensorflow:processing img 2350687181.jpg
./data/flickr30k-images/2350687181.jpg
INFO:tensorflow:processing img 235074044.jpg
./data/flickr30k-images/235074044.jpg
INFO:tensorflow:processing img 2350843831.jpg
./data/flickr30k-images/2350843831.jpg
INFO:tensorflow:processing img 2351153998.jpg
./data/flickr30k-images/2351153998.jpg
INFO:tensorflow:processing img 2351479551.jpg
./data/flickr30k-images/2351479551.jpg
INFO:tensorflow:processing img 2351694146.jpg
./data/flickr30k-images/2351694146.jpg
INFO:tensorflow:processing img

./data/flickr30k-images/2363419943.jpg
INFO:tensorflow:processing img 2363520556.jpg
./data/flickr30k-images/2363520556.jpg
INFO:tensorflow:processing img 2363540508.jpg
./data/flickr30k-images/2363540508.jpg
INFO:tensorflow:processing img 2363918218.jpg
./data/flickr30k-images/2363918218.jpg
INFO:tensorflow:processing img 2364096157.jpg
./data/flickr30k-images/2364096157.jpg
INFO:tensorflow:processing img 2364394224.jpg
./data/flickr30k-images/2364394224.jpg
INFO:tensorflow:processing img 2364728258.jpg
./data/flickr30k-images/2364728258.jpg
INFO:tensorflow:processing img 236474697.jpg
./data/flickr30k-images/236474697.jpg
INFO:tensorflow:processing img 236476706.jpg
./data/flickr30k-images/236476706.jpg
INFO:tensorflow:processing img 2364774105.jpg
./data/flickr30k-images/2364774105.jpg
INFO:tensorflow:processing img 2364942064.jpg
./data/flickr30k-images/2364942064.jpg
INFO:tensorflow:processing img 2364982285.jpg
./data/flickr30k-images/2364982285.jpg
INFO:tensorflow:processing img

./data/flickr30k-images/2375981973.jpg
INFO:tensorflow:processing img 2376252671.jpg
./data/flickr30k-images/2376252671.jpg
INFO:tensorflow:processing img 2376413845.jpg
./data/flickr30k-images/2376413845.jpg
INFO:tensorflow:processing img 237650533.jpg
./data/flickr30k-images/237650533.jpg
INFO:tensorflow:processing img 2376694294.jpg
./data/flickr30k-images/2376694294.jpg
INFO:tensorflow:processing img 2376878930.jpg
./data/flickr30k-images/2376878930.jpg
INFO:tensorflow:processing img 23770160.jpg
./data/flickr30k-images/23770160.jpg
INFO:tensorflow:processing img 237726613.jpg
./data/flickr30k-images/237726613.jpg
INFO:tensorflow:processing img 2377460540.jpg
./data/flickr30k-images/2377460540.jpg
INFO:tensorflow:processing img 2377496811.jpg
./data/flickr30k-images/2377496811.jpg
INFO:tensorflow:processing img 23775702.jpg
./data/flickr30k-images/23775702.jpg
INFO:tensorflow:processing img 2378055886.jpg
./data/flickr30k-images/2378055886.jpg
INFO:tensorflow:processing img 2378127

./data/flickr30k-images/2390400553.jpg
INFO:tensorflow:processing img 239051548.jpg
./data/flickr30k-images/239051548.jpg
INFO:tensorflow:processing img 2390551509.jpg
./data/flickr30k-images/2390551509.jpg
INFO:tensorflow:processing img 2390778197.jpg
./data/flickr30k-images/2390778197.jpg
INFO:tensorflow:processing img 2391015435.jpg
./data/flickr30k-images/2391015435.jpg
INFO:tensorflow:processing img 2391035976.jpg
./data/flickr30k-images/2391035976.jpg
INFO:tensorflow:processing img 2391048897.jpg
./data/flickr30k-images/2391048897.jpg
INFO:tensorflow:processing img 2391094555.jpg
./data/flickr30k-images/2391094555.jpg
INFO:tensorflow:processing img 239112510.jpg
./data/flickr30k-images/239112510.jpg
INFO:tensorflow:processing img 2391152358.jpg
./data/flickr30k-images/2391152358.jpg
INFO:tensorflow:processing img 2391269207.jpg
./data/flickr30k-images/2391269207.jpg
INFO:tensorflow:processing img 2391442557.jpg
./data/flickr30k-images/2391442557.jpg
INFO:tensorflow:processing img

./data/flickr30k-images/2402744031.jpg
INFO:tensorflow:processing img 2402762193.jpg
./data/flickr30k-images/2402762193.jpg
INFO:tensorflow:processing img 2402793046.jpg
./data/flickr30k-images/2402793046.jpg
INFO:tensorflow:processing img 2403078014.jpg
./data/flickr30k-images/2403078014.jpg
INFO:tensorflow:processing img 2403376030.jpg
./data/flickr30k-images/2403376030.jpg
INFO:tensorflow:processing img 2403532277.jpg
./data/flickr30k-images/2403532277.jpg
INFO:tensorflow:processing img 2403544744.jpg
./data/flickr30k-images/2403544744.jpg
INFO:tensorflow:processing img 2403610816.jpg
./data/flickr30k-images/2403610816.jpg
INFO:tensorflow:processing img 2403832405.jpg
./data/flickr30k-images/2403832405.jpg
INFO:tensorflow:processing img 2404046478.jpg
./data/flickr30k-images/2404046478.jpg
INFO:tensorflow:processing img 2404173027.jpg
./data/flickr30k-images/2404173027.jpg
INFO:tensorflow:processing img 2404406118.jpg
./data/flickr30k-images/2404406118.jpg
INFO:tensorflow:processing